In [ ]:
from requests_html import HTMLSession
import requests
import wptools
from tqdm import tqdm
from mwviews.api import PageviewsClient
import pandas as pd
import plotly
import plotly.express as px
import json
from furl import furl

pd.set_option('display.max_columns', None)

# Get links

In [ ]:
seed = 'List of pharaohs'

page = wptools.page(seed, lang='en')

In [ ]:
page.get()

In [ ]:
links = page.data['links']

In [ ]:
len(page.data['links'])

# Get features of links

In [ ]:

df_list = []

for link in tqdm(set(links)):
    try:
        p = wptools.page(link, lang='en')
        p.get()
        df_list.append(p.data)
    except Exception as e:
        print('ERROR', link, str(e))

In [ ]:
df = pd.DataFrame(df_list)
df.shape

In [ ]:
# Sends a descriptive User-Agent header with every request
pc = PageviewsClient(user_agent="person@organization.org Selfie, Cat, and Dog analysis")

#p.article_views('en.wikipedia', ['Selfie', 'Cat', 'Dog'])
#p.project_views(['ro.wikipedia', 'de.wikipedia', 'commons.wikimedia'])
#p.top_articles('en.wikipedia', limit=10)

#articles = df.query('what == "human"')['title'].tolist()

df['pageviews'] = list(pc.article_views('en.wikipedia', df['title'].tolist(), granularity='monthly', start='20191001', end='20191031').values())[0].values()
# = list(out.values())[0]
# Feel free to add your own features in pull requests!

In [ ]:
df.to_pickle(f'../results/{seed}.pkl')
df.shape

In [ ]:
#df.memory_usage(deep=True).sort_values()

In [ ]:
to_drop =  ['parsetree', 'wikitext', 'links', 'extext', 'extract', 'WARNINGS', 'exrest', 'exhtml']
df.drop(to_drop, axis=1).to_csv('../results/df.csv', index=False)

In [ ]:
df.drop(to_drop, axis=1).to_excel('../results/df.xlsx', index=False)

# Superrychla varianta

In [ ]:
# stiahne ubodnu stranku
r = HTMLSession().get('https://en.wikipedia.org/wiki/Index_of_ancient_Egypt-related_articles')

# scrapne pozadovane linky
css = "div#mw-content-text > div.mw-parser-output > div[class='hlist'] a"
titles = [furl(a.attrs['href']).asdict()['path']['segments'][-1] for a in r.html.find(css)]
print(len(titles))

# stiahne pageviewsy
pws = list(PageviewsClient(user_agent="person@organization.org Selfie, Cat, and Dog analysis").article_views('en.wikipedia', titles, granularity='monthly', start='20191001', end='20191031').values())[0].values()

# pripravi strukturu pre JS
data = [{'Name':x, 'Count':y, 'url':f'https://en.wikipedia.org/wiki/{x}'} for x, y in zip(titles, pws)]
with open('../results/data.txt', 'w') as f:
    f.write(json.dumps(data))

# DS

In [ ]:
df = pd.read_pickle('../results/List of pharaohs.pkl')

In [ ]:
tops = df['what'].value_counts().head(20).index
df['what_top'] = [x if x in tops else 'other' for x in df['what']] 

In [ ]:
fig = px.scatter(
    df, 
    x='length', y='watchers', 
    color = 'what_top',
    hover_data=['title'])

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.show()

In [ ]:
plotly.offline.plot(fig, filename = '../results/plotly.html', auto_open=False)

In [ ]:
#df.query('what == "human"')[['url', 'length', 'watchers']].sample(10)

In [ ]:
# vystup pre d3.js bubble chart https://bl.ocks.org/alokkshukla/3d6be4be0ef9f6977ec6718b2916d168
list(df[['pageviews', 'title', 'url']].rename(columns={'pageviews':'Count', 'title':'Name'}).T.to_dict().values())

# References

# Trash

In [ ]:
r = HTMLSession().get('https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States')





In [ ]:
css = "div#mw-content-text > div.mw-parser-output tr > td > b > big > a"
r.html.find(css)

In [ ]:
print(len(r.html.find(css)))

from furl import furl
titles = [furl(a.attrs['href']).asdict()['path']['segments'][-1] for a in r.html.find(css)]
len(titles)

In [ ]:
pws = list(PageviewsClient(user_agent="person@organization.org Selfie, Cat, and Dog analysis").article_views('en.wikipedia', titles, granularity='monthly', start='20191001', end='20191031').values())[0].values()

data = [{'Name':x, 'Count':y, 'url':f'https://en.wikipedia.org/wiki/{x}'} for x, y in zip(titles, pws)]

import json



In [ ]:
with open('../results/data.txt', 'w') as f:
    f.write(json.dumps(data))

In [ ]:
pd.concat(data).drop_duplicates().shape